In [ ]:
from exp.utils import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [ ]:
sigmoid = NN.Sigmoid()

In [ ]:
model_name = "py_test_01"

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU!")
else:
    device = torch.device("cpu")
    print("Using the CPU!")

In [ ]:
bs = 16

In [ ]:
labels = get_labels()

In [ ]:
train_df, valid_df, test_df = get_dataframes(small=True)

In [ ]:
train_label = train_df[labels].values
pos_weights = compute_positive_class_weigths(train_label)

In [ ]:
train_tfs = transforms.Compose([
    transforms.Normalize([SUBSET_MEAN]*3, [SUBSET_STD]*3),
    transforms.RandomHorizontalFlip(p=0.5)
])
test_tfs = transforms.Compose([
    transforms.Normalize([SUBSET_MEAN]*3, [SUBSET_STD]*3)
    
])

In [ ]:
train_ds = CRX8_Data(train_df, get_image_path(), labels, transforms=train_tfs)
valid_ds = CRX8_Data(valid_df, get_image_path(), labels, transforms=test_tfs)
test_ds  = CRX8_Data(test_df , get_image_path(), labels, transforms=test_tfs)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)

In [ ]:
criterion = NN.BCEWithLogitsLoss(pos_weight=pos_weights.to(device), reduction="sum")

In [ ]:
import torchvision.models as models
net = models.resnet18(pretrained=True)
net.fc = NN.Linear(512, len(get_labels()))
net = net.to(device)

In [ ]:
optimizer = torch.optim.Adam(net.parameters(),lr=1e-1)

In [ ]:
#logs,losses = find_lr(net, train_dl, optimizer, criterion, device=device)
#plt.plot(logs[10:-5],losses[10:-5])

In [ ]:
lr = 1e-3

In [ ]:
dataloaders = {
    "train": train_dl,
    "val": valid_dl,
    "test": test_dl
}

In [ ]:
epochs = 5
num_steps = len(train_dl) * epochs
lr_range = (lr / 10, lr)


scheduler = OneCycleLR(optimizer, num_steps, lr_range)

In [ ]:
steps, lrs, moms = [], [], []

scheduler.num_steps, scheduler.num_cycle_steps

for i in range(scheduler.num_steps*2):
    steps.append(i)
    lrs.append(scheduler.get_lr())
    moms.append(scheduler.get_momentum())
    scheduler.step()
    if i == scheduler.num_steps: scheduler.reset()

#scheduler.get_lr(), scheduler.get_momentum() 

In [ ]:
plt.plot(steps, lrs)

In [ ]:
plt.plot(steps, moms)

In [ ]:
class Logger:
    def __init__(self):
        self.state = {}
    
    def get_name(self, phase, name): return f"{phase}_{name}"
    
    def add_metric(self, name, phases=["train", "val"]):
        for phase in phases: self.state[self.get_name(phase, name)] = []

    def add_value(self, name, value, phase):
        self.state[self.get_name(phase, name)].append(value)
    
    def _plot_values(self, name):
        train_name = self.get_name("train", name)
        val_name = self.get_name("val", name)
        plt.plot(
            list(range(len(self.state[train_name]))), 
            self.state[train_name]);
        plt.plot(
            list(range(len(self.state[val_name]))), 
            self.state[val_name]);
    
    def plot_losses(self):
        self._plot_values("loss")
    
    def plot_acc(self):
        self._plot_values("acc")
    
    def plot_auroc(self, phase="val"):
        name = self.get_name(phase, "auroc")
        df = self.state[name][0][0]
        for data, _ in self.state[name]:
            df = pd.concat([df, data], axis=1)
        return add_chexnet(df)

In [ ]:
logger = Logger()
logger.add_metric("loss")
logger.add_metric("acc")
logger.add_metric("auroc")

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
def auroc_score(y_hat, y, model_name):
    labels = get_labels()
    score = {l:[] for l in labels}
    label_thresholds = {}
    for l_idx, l in enumerate(labels):
        fpr, tpr, thresholds = roc_curve(y[:, l_idx], y_hat[:, l_idx])
        auc_value = auc(fpr, tpr)
        score[l].append(auc_value)
        label_thresholds[l] = thresholds[np.argmax(tpr - fpr)]
    df = pd.DataFrame(score.values(), index=score.keys(), columns=[model_name])
    return df, label_thresholds

In [ ]:
#export
def train_model(model, criterion, optimizer, scheduler, dataloaders, logger, 
                num_epochs=25, device=torch.device("cuda:0")):
    since = time.time()
    model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    running_acc = 0
    rabeta = 0.9

    for epoch in tqdm(range(num_epochs)):
        for phase in ['train', 'val']:
            if phase == 'train': model.train()  
            else: model.eval()   

            running_loss = 0.0
            running_corrects = 0
            
            y_hat, y = [], []

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    sigmoid_outputs = sigmoid(outputs)
                    thresholded = sigmoid_outputs >= 0.5
                    loss = criterion(outputs, labels)
                
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    y_hat = [*y_hat, *sigmoid_outputs.cpu().detach().numpy()]
                    y = [*y, *labels.cpu().detach().numpy()]

                running_loss += loss.item() * inputs.size(0)
                running_corrects += (thresholded == labels).sum() / labels.numel()
                
            if phase == 'train': scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_auroc = auroc_score(np.array(y_hat), np.array(y), f"{model_name}_e{epoch}")
            
            logger.add_value("loss",  epoch_loss,  phase)
            logger.add_value("acc",   epoch_acc,   phase)
            logger.add_value("auroc", epoch_auroc, phase)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        scheduler.reset()

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

In [ ]:
net = train_model(net, criterion, optimizer, scheduler, dataloaders, logger,
                  num_epochs=epochs, device=device)

In [ ]:
for k,v in logger.state.items():
    print(k)

In [ ]:
logger.plot_losses()

In [ ]:
type(logger.state["train_auroc"][0])

In [ ]:
df = logger.state["train_auroc"][0][0]
for data, _ in logger.state["train_auroc"]:
    df = pd.concat([df, data], axis=1)
add_chexnet(df)

In [ ]:
plt.plot(
    list(range(len(logger.state["train_loss"]))), 
    logger.state["train_loss"])
plt.plot(
    list(range(len(logger.state["val_loss"]))), 
    logger.state["val_loss"])

In [ ]:
plt.plot(
    list(range(len(logger.state["train_loss"]))), 
    logger.state["train_loss"])
plt.plot(
    list(range(len(logger.state["val_loss"]))), 
    logger.state["val_loss"])

In [ ]:
def evaluate_model(model, dl, criterion, model_name):
    y_hat, y = [], []
    test_loss = 0.
    
    for X_batch, y_batch in tqdm(dl):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        logits = model(X_batch)
        
        criterion(logits, y_batch).item()
        test_loss += criterion(logits, y_batch).item()
        
        y_hat_batch = sigmoid(logits).cpu().detach().numpy()
        y_hat = [*y_hat, *y_hat_batch]
        y = [*y, *y_batch.cpu().detach().numpy()]
    
    test_loss /= len(dl.dataset)
    test_auroc, label_thresholds = auroc_score(np.array(y_hat), np.array(y), model_name)
    
    return test_loss, test_auroc, label_thresholds

In [ ]:
test_loss, test_auroc, label_thresholds = evaluate_model(net,  test_dl, criterion, model_name)

In [ ]:
test_loss*32*8

In [ ]:
add_chexnet(test_auroc)

In [ ]:
test_auroc

In [ ]:
label_thresholds

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
y_hat = torch.rand((899, 14))
y = torch.rand((899, 14)) > 0.5

In [ ]:
multilabel_confusion_matrix(y_hat > 0.5, y)

In [ ]:
auroc_score(y_hat, y)

In [ ]:
a = [[-1, 0],[1, 2]]
b = [[3, 4]]

a = [*a, *b]
a